In [4]:
import numpy as np
from astropy.io import fits
from pprint import pprint

In [6]:
wco = fits.open(r'e:\drive\Koeln\COBE__data\PVmap\fit_files\Wco_DHT2001.fits')
pprint(wco[0].header)

SIMPLE  =                    T  / Standard FITS format                          
BITPIX  =                   16  / Bits per pixel                                
NAXIS   =                    2  / Number of axes                                
NAXIS1  =                 2961  / Number pixels on axis 1                       
NAXIS2  =                  561  / Number pixels on axis 2                       
CTYPE1  = 'GLON-CAR'            / axis 1 coord type                             
CRVAL1  =         0.000000E+00 / coord value at CRPIX1                          
CDELT1  =        -1.250000E-01 / pixel spacing for axis 1                       
CRPIX1  =          1481.000000  / ref pixel for axis 1                          
CTYPE2  = 'GLAT-CAR'            / axis 2 coord type                             
CRVAL2  =         0.000000E+00 / coord value at CRPIX2                          
CDELT2  =         1.250000E-01 / pixel spacing for axis 2                       
CRPIX2  =           281.0000

 [astropy.io.fits.verify]


In [11]:
cogal = fits.open(r'e:\drive\Koeln\COBE__data\PVmap\fit_files\COGAL_all_interp_all.fits')
pprint(cogal[0].header)
np.where(np.isnan(cogal[0].data))[0].size

SIMPLE  =                    T  / Standard FITS format                          
BITPIX  =                   16  / Bits per pixel                                
NAXIS   =                    3  / Number of axes                                
NAXIS1  =                  493  / Number pixels on axis 1                       
NAXIS2  =                 2881  / Number pixels on axis 2                       
NAXIS3  =                  481  / Number pixels on axis 3                       
CTYPE1  = 'VELO-LSR'            / axis 1 coord type                             
CRVAL1  =        -3.198000E+02  / coord value at CRPIX1                         
CDELT1  =         1.300000E+00  / linear pixel spacing axis 1                   
CRPIX1  =             1.000000  / ref pixel for axis 1                          
CTYPE2  = 'GLON-CAR'            / axis 2 coord type                             
CRVAL2  =         1.800000E+02  / coord value at CRPIX2                         
CDELT2  =        -1.250000E-

626332143

In [8]:
cogal = fits.open(r'e:\drive\Koeln\COBE__data\PVmap\fit_files\COGAL_deep_interp_5degree.fits')
pprint(cogal[0].header)

SIMPLE  =                    T  / Standard FITS format                          
BITPIX  =                   16  / Bits per pixel                                
NAXIS   =                    3  / Number of axes                                
NAXIS1  =                  493  / Number pixels on axis 1                       
NAXIS2  =                 2881  / Number pixels on axis 2                       
NAXIS3  =                   81  / Number pixels on axis 3                       
CTYPE1  = 'VELO-LSR'            / axis 1 coord type                             
CRVAL1  =        -3.198000E+02  / coord value at CRPIX1                         
CDELT1  =         1.300000E+00  / linear pixel spacing axis 1                   
CRPIX1  =             1.000000  / ref pixel for axis 1                          
CTYPE2  = 'GLON-CAR'            / axis 2 coord type                             
CRVAL2  =         1.800000E+02  / coord value at CRPIX2                         
CDELT2  =        -1.250000E-

In [ ]:
from scipy.interpolate import interp2d
from copy import copy

def model_selection(path=r'c:\users\cyani\projects\pdr\KT3_history\MilkyWay', resolution=400, 
                    f_clump=[0.5, 1.0, 2.0, 4.0], f_interclump=[1.0, 2.0], 
                    f_density=[0.5, 1.0, 2.0, 4.0], f_uv=[10, 100], transition='CO 1'):
    # This function will compare the Milky Way models to the position-velocity observations depending on the
    #  dataset. It will utilise the sightlines of the dataset.
    # Currently, this will look solely at the galactic plane.
    
    if transition == 'CO 1':
        obs = fits.open('/media/yanitski/4,0 TB Hard Disk/yanitski/projects/pdr/observational_data/MilkyWay/COGAL/COGAL_all_interp_all.fits')
        obs_data = obs[0].data.sum(0)
        lon = np.linspace(obs[0].header['CRVAL2']-obs[0].header['CDELT2']*obs[0].header['CRPIX2'],
                          obs[0].header['CRVAL2']+obs[0].header['CDELT2']*obs[0].header['CRPIX2'],
                          num=obs[0].header['NAXIS2'])
        vel = np.linspace(obs[0].header['CRVAL1']-obs[0].header['CDELT1']*obs[0].header['CRPIX1'],
                          obs[0].header['CRVAL1']+obs[0].header['CDELT1']*obs[0].header['CRPIX1'],
                          num=obs[0].header['NAXIS1'])
        obs_error = 0.5
        
    
    elif transition == '13CO 2':
        print('This transition usiing SEDIGISM is still pending...')
#         obs = fits.open(r'e:\drive\Koeln\COBE__data\PVmap\fit_files\COGAL_all_interp_all.fits')
    
    chi2_grid = []
    likelihood_grid = []
    
    # Loop through the different models
    for fcl in f_clump:
        for ficl in f_interclump:
            for fden in f_density:
                for fuv in f_uv:
                    
                    # Open the model
                    dir_model = '/r{}_cm{}-{}_d{}_uv{}/channel_intensity'.format(resolution, fcl, ficl, fden, fuv)
                    model = fits.open(path+dir_model)
                    
                    # Locate the species transition
                    i_spec = np.where(model[1].header['species'].split(', ') == transition)[0][0]
                    
                    # Locate the galactic disk
                    if model[1].header['CRPIX3'] == 0:
                        i_lat = np.floor(model[1].header['NAXIS3']/2).astype(np.int)
                    else:
                        lat = np.linspace(model[1].header['CRVAL3']-model[1].header['CDELT3']*model[1].header['CRPIX3'],
                                          model[1].header['CRVAL3']+model[1].header['CDELT3']*model[1].header['CRPIX3'],
                                          num=model[1].header['NAXIS3'])
                        i_lat = np.where(np.abs(lat-0) == np.abs(lat-0).min())[0][0]
                    
                    # Create arrays for the longitude and velocity axes
                    lon_model = np.linspace(model[1].header['CRVAL2']-model[1].header['CDELT2']*model[1].header['CRPIX2'],
                                            model[1].header['CRVAL2']+model[1].header['CDELT2']*model[1].header['CRPIX2'],
                                            num=model[1].header['NAXIS2'])
                    vel_model = np.linspace(model[1].header['CRVAL4']-model[1].header['CDELT4']*model[1].header['CRPIX4'],
                                            model[1].header['CRVAL4']+model[1].header['CDELT4']*model[1].header['CRPIX4'],
                                            num=model[1].header['NAXIS4'])
                    
                    # Interpolate at the observational axes
                    f_model_interp = interp2d(vel_model, lon_model, model[1].data[:,i_lat,:,i_spec], kind='cubic')
                    model_interp = f_model_interp(vel, lon)
                    
                    # Calculate the chi**2 and overall likelihood
                    chi2 = (obs_data*model_interp)**2/obs_error**2
                    likelihood = np.exp(-chi2-2)/np.sqrt(2*np.pi*obs_error)
                    
                    chi2_grid.append(chi2.prod())
                    likelihood_grid.append(likelihood.prod())